In [1]:
import os
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import wordpunct_tokenize
# from nltk.tokenize import sent_tokenize (Tokenization)
from nltk.probability import FreqDist
from nltk.metrics import ConfusionMatrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /Users/joao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /Users/joao/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
%run NLPha

[nltk_data] Downloading package stopwords to /Users/joao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /Users/joao/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
#from NLPha import _remove_url, _remove_regex, _replace_emoticons, _tokenize_text 
#from NLPha import _apply_standardization, _remove_stopwords, _apply_stemmer, _untokenize_text, _get_text_cloud
#from NLPha import _get_freq_dist_list, _get_accuracy


# get data

In [3]:
dftreino =  pd.read_csv('data.csv', delimiter=';')
xtreino = list(dftreino.text)

# preprocess

In [4]:
xtreino = _remove_url(xtreino)

regex_pattern = '@[\w]*'

xtreino = _remove_regex(xtreino, regex_pattern)

emoticon_list = {':))': 'positive_emoticon', ':)': 'positive_emoticon', ':D': 'positive_emoticon', ':(': 'negative_emoticon', ':((': 'negative_emoticon', '8)': 'neutral_emoticon'}

xtreino = _replace_emoticons(xtreino, emoticon_list)

xtreino_tokens = _tokenize_text(xtreino)

xtreino_tokens = _apply_standardization(xtreino_tokens, std_list)

xtreino_tokens = _remove_stopwords(xtreino_tokens, stopword_list)

X = _untokenize_text(xtreino_tokens)

In [5]:
tweets = dftreino.text
classes = dftreino.isFavorable

# organize for input 

In [6]:
vocabulary = []
index = dict()
encoded_tweets = []

for tweet in xtreino_tokens:
    encoded_tweet = []

    for word in tweet:
        if word not in index:
            vocabulary.append(word)
            index[word] = len(vocabulary) - 1

    # encoding:
    for word in tweet:
        encoded_tweet.append(index[word])
    encoded_tweets.append(encoded_tweet)
    #print(encoded_review)

In [7]:
X = encoded_tweets
y = classes

In [8]:
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
import numpy as np

print("testX.shape:",np.array(X_test).shape) #2500个list.每个list代表一句话
print("testY.shape:",np.array(y_test).shape) #2500个label
print("testX[0]:",X_test[0]) #[17, 25, 10, 406, 26, 14, 56, 61, 62, 323, 4]
print("testY[0]:",y_test[0]) #0

testX.shape: (4026,)
testY.shape: (4026,)
testX[0]: [646, 38, 39, 5703, 147, 497, 388, 1937, 5327, 6484, 170, 280, 498, 435, 248, 1399, 182, 776, 55, 21, 19431, 21, 20027]
testY[0]: 1


<ipython-input-26-23f7b34a0095>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print("testX.shape:",np.array(X_test).shape) #2500个list.每个list代表一句话


# Sentiment analysis with CNN

In [9]:
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:


# 2.Data preprocessing
# Sequence padding
trainX = pad_sequences(trainX, maxlen=100, value=0.) #padding to max length
testX = pad_sequences(testX, maxlen=100, value=0.)   #padding to max length
# Converting labels to binary vectors
trainY = to_categorical(trainY, nb_classes=2) #y as one hot
testY = to_categorical(testY, nb_classes=2)   #y as one hot

# 3.Building convolutional network
#(shape=None, placeholder=None, dtype=tf.float32,data_preprocessing=None, data_augmentation=None,name="InputData")
network = input_data(shape=[None, 100], name='input') #[None, 100] `input_data` is used as a data entry (placeholder) of a network. This placeholder will be feeded with data when training
network = tflearn.embedding(network, input_dim=10000, output_dim=128) #[None, 100,128].embedding layer for a sequence of ids. network: Incoming 2-D Tensor. input_dim: vocabulary size, oput_dim:embedding size
         #conv_1d(incoming,nb_filter,filter_size)
branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2") # [batch_size, new steps1, nb_filters]. padding:"VALID",only ever drops the right-most columns
branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2") # [batch_size, new steps2, nb_filters]
branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2") # [batch_size, new steps3, nb_filters]
network = merge([branch1, branch2, branch3], mode='concat', axis=1) # merge a list of `Tensor` into a single one.===>[batch_size, new steps1+new step2+new step3, nb_filters]
network = tf.expand_dims(network, 2) #[batch_size, new steps1+new step2+new step3,1, nb_filters] Inserts a dimension of 1 into a tensor's shape
network = global_max_pool(network) #[batch_size, pooled dim]
network = dropout(network, 0.5) #[batch_size, pooled dim]
network = fully_connected(network, 2, activation='softmax') #matmul([batch_size, pooled_dim],[pooled_dim,2])---->[batch_size,2]
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(trainX, trainY, n_epoch = 5, shuffle=True, validation_set=(testX, testY), show_metric=True, batch_size=32)
print("ended...")